<a href="https://colab.research.google.com/github/GYVVishnu77/1M1B_Project_Urban_Mobility/blob/main/traffic_mgmt_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pypdf import PdfReader


print("✅ Libraries loaded")


# ===============================
# Upload PDFs
# ===============================

uploaded = files.upload()
pdf_files = list(uploaded.keys())

texts = []

for pdf in pdf_files:
    reader = PdfReader(pdf)

    for page in reader.pages:
        content = page.extract_text()
        if content:
            texts.append(content)

print("Pages extracted:", len(texts))


# ===============================
# Chunking (LIGHTWEIGHT)
# ===============================

def chunk_text(text, size=700, overlap=150):
    chunks = []

    start = 0
    while start < len(text):
        end = start + size
        chunks.append(text[start:end])
        start += size - overlap

    return chunks


chunks = []

for t in texts:
    chunks.extend(chunk_text(t))

print("Chunks created:", len(chunks))


# ===============================
# Embeddings (VERY LIGHT)
# ===============================

print("Loading embedding model...")

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-MiniLM-L3-v2"
)

embeddings = embed_model.encode(chunks)

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("✅ Vector DB ready!")


# ===============================
# Load SMALL LLM
# ===============================

print("Loading lightweight LLM...")

generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_length=256
)

print("✅ LLM ready!")


# ===============================
# Retrieval Function
# ===============================

def retrieve(query, k=2):

    q_embedding = embed_model.encode([query])

    distances, indices = index.search(np.array(q_embedding), k)

    results = [chunks[i] for i in indices[0]]

    return results


# ===============================
# RAG Ask Function
# ===============================

def ask(query):

    contexts = retrieve(query)

    context_text = "\n\n".join(contexts)

    prompt = f"""
Answer the question using ONLY the context below.

Context:
{context_text}

Question:
{query}
"""

    result = generator(prompt)[0]["generated_text"]

    print("\nANSWER:\n")
    print(result)

    print("\n--- Sources pulled from uploaded PDFs ---\n")


# ===============================
# Interactive Loop
# ===============================

print("\n🎉 URBAN INTELLIGENCE AGENT READY!")
print("Type 'exit' to stop.\n")

while True:

    q = input("Ask a question: ")

    if q.lower() == "exit":
        break

    ask(q)


✅ Libraries loaded


Saving mobility.pdf to mobility.pdf
Saving sustainable.pdf to sustainable.pdf
Saving sustainable_1.pdf to sustainable_1.pdf
Pages extracted: 60
Chunks created: 249
Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


✅ LLM ready!

🎉 URBAN INTELLIGENCE AGENT READY!
Type 'exit' to stop.

Ask a question: how to manage traffic in bangalore 

ANSWER:

PT

--- Sources pulled from uploaded PDFs ---

Ask a question: exit


In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers pypdf


In [ ]:
from google.colab import files
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pypdf import PdfReader


print("✅ Libraries loaded successfully")


# =====================================
# Upload PDFs
# =====================================

print("\nUpload ONLY 2–3 PDFs for best performance.\n")

uploaded = files.upload()
pdf_files = list(uploaded.keys())

texts = []

for pdf in pdf_files:
    reader = PdfReader(pdf)

    for page in reader.pages:
        content = page.extract_text()
        if content:
            texts.append(content)

print("✅ Pages extracted:", len(texts))


# =====================================
# Smart Chunking
# =====================================

def chunk_text(text, size=700, overlap=150):
    chunks = []
    start = 0

    while start < len(text):
        end = start + size
        chunks.append(text[start:end])
        start += size - overlap

    return chunks


chunks = []

for t in texts:
    chunks.extend(chunk_text(t))

print("✅ Chunks created:", len(chunks))


# =====================================
# Embeddings (LOW RAM)
# =====================================

print("\nLoading embedding model...")

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-MiniLM-L3-v2"
)

embeddings = embed_model.encode(
    chunks,
    show_progress_bar=True
)

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("✅ Vector database ready!")


# =====================================
# Load Lightweight LLM
# =====================================

print("\nLoading lightweight LLM...")

generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_length=512,
    temperature=0.1
)

print("✅ LLM ready!")


# =====================================
# Retrieval Function
# =====================================

def retrieve(query, k=3):

    q_embedding = embed_model.encode([query])

    distances, indices = index.search(
        np.array(q_embedding), k
    )

    results = [chunks[i] for i in indices[0]]

    return results


# =====================================
# Intelligent RAG Function
# =====================================

def ask(query):

    contexts = retrieve(query)

    context_text = "\n\n".join(contexts)

    prompt = f"""
You are an expert in urban planning and smart cities.

Using ONLY the context below, generate a structured and detailed answer.

RULES:
- Do NOT give one-word answers.
- Do NOT abbreviate.
- Explain clearly.
- If information is missing, say so.

Format your answer as:

Problem:
Solutions:
Expected Impact:

Context:
{context_text}

Question:
{query}

Answer:
"""

    result = generator(prompt)[0]["generated_text"]

    print("\n✅ ANSWER:\n")
    print(result)

    print("\n📚 Retrieved Context Preview:\n")
    print(context_text[:500])   # Debug preview

    print("\n" + "="*65 + "\n")


# =====================================
# Interactive Agent
# =====================================

print("\n🎉 URBAN INTELLIGENCE AGENT READY!")
print("Type 'exit' to stop.\n")

while True:

    q = input("Ask a question: ")

    if q.lower() == "exit":
        break

    ask(q)


✅ Libraries loaded successfully

Upload ONLY 2–3 PDFs for best performance.



Saving mobility.pdf to mobility (1).pdf
Saving sustainable.pdf to sustainable (1).pdf
✅ Pages extracted: 56
✅ Chunks created: 194

Loading embedding model...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Vector database ready!

Loading lightweight LLM...


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ LLM ready!

🎉 URBAN INTELLIGENCE AGENT READY!
Type 'exit' to stop.

Ask a question: What strategies can cities use to reduce traffic congestion

✅ ANSWER:

congestion pricing, smart traffic management, and intelligent transport systems (ITS)

📚 Retrieved Context Preview:

 
and sustainable 
transport options to 
all citizens.
•	 T o reduce congestion 
on the roads and 
enhance the overall 
mobility of people 
and goods within 
thecity.
•	 T o promote the 
use of public 
transportation and 
non-motorised 
modes of transport, 
such as walking 
and cycling, to 
reduce reliance on 
private vehicles 
and minimise 
environmental 
impact.
•	 T o improve the 
accessibility of 
different parts of 
the city to ensure 
equitable access 
to transportation 
services.
•	 T o e


